In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import pylab

### Load Data

In [12]:
X = []
Y = []
df = np.array(pd.read_csv("train_img.csv"))
for i in df:
    X.append(np.array(Image.open(f"train_images/train_images/{i[0]}").resize((128,128))))
    Y.append(i[1])

In [13]:
Y_mapping = {}
l = 0
for i in range(len(Y)):
    try:
        tmp = Y_mapping[Y[i]]
        Y[i] = tmp
    except:
        Y_mapping[Y[i]] = l
        Y[i] = l
        l+=1

In [16]:
for i in range(len(Y)):
    tmp = np.zeros_like([0]*(l+1))
    tmp[Y[i]] = 1
    Y[i] = tmp

### Model

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128,128,3)),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(256,(3,3),activation="relu"),
    tf.keras.layers.Conv2D(256,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(len(Y_mapping.keys())+1, activation="relu"),
    tf.keras.layers.Dense(len(Y_mapping.keys())+1, activation="softmax")
])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_81 (Conv2D)          (None, 126, 126, 64)      1792      
                                                                 
 conv2d_82 (Conv2D)          (None, 124, 124, 64)      36928     
                                                                 
 conv2d_83 (Conv2D)          (None, 122, 122, 64)      36928     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 61, 61, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_84 (Conv2D)          (None, 59, 59, 128)       73856     
                                                                 
 conv2d_85 (Conv2D)          (None, 57, 57, 128)       147584    
                                                     

In [39]:
X = np.array(X)
Y = np.array(Y)

In [40]:
model.fit(X, Y, epochs=10)

Epoch 1/10
 35/292 [==>...........................] - ETA: 21:41 - loss: 5.1793